In [ ]:
import os
import gym
import numpy as np

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.callbacks import BaseCallback

from Enviroment import Enviroment
from gym import spaces
import cv2
from tqdm import tqdm

from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import torch.nn.functional as F
import torch
from stable_baselines3 import PPO

In [ ]:
torch.backends.cudnn.benchmark = True

## Callback class

In [ ]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """
    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model_PPO')
        self.best_mean_reward = -np.inf
        self.rew_len = []
        self.n_games = 0

    def _on_step(self) -> bool:
        
        if self.n_games < len(model.ep_info_buffer):
            self.n_games+=1
            self.rew_len.append(model.ep_info_buffer[-1]['r'])
            
            if self.best_mean_reward < np.mean(self.rew_len[-100:]):
                self.best_mean_reward = np.mean(self.rew_len[-100:])
                self.model.save(self.save_path)

        return True

## Neural network class

In [ ]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Dict, features_dim: int = 518):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        
        
        extractors = {}
        
        for key, subspace in observation_space.spaces.items():
            if key == "img":
        
                n_input_channels = observation_space[key].shape[0]
            
                extractors[key] = nn.Sequential(

                nn.Conv2d(n_input_channels, 32, 2),
                nn.MaxPool2d(2, 2),
                nn.Conv2d(32, 64, 2),
                nn.MaxPool2d(2, 2),

                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(4, 4),
                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(2, 2),
                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(2, 2),
                ResBlock(n_filters=64, kernel_size=2), 
                nn.MaxPool2d(2, 2),
                
                nn.Conv2d(64, 128, 2),
                nn.Flatten())
                    
            elif key == "posRobot":
                
                n_input_channels = observation_space[key].shape[0]
                
                extractors[key] = nn.Sequential(nn.Linear(n_input_channels, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 3))
                    
            elif key == "target":
                            
                n_input_channels = observation_space[key].shape[0]
                    
                extractors[key] = nn.Sequential(nn.Linear(n_input_channels, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 3))
                
        self.extractors = nn.ModuleDict(extractors)

    def forward(self, observations: th.Tensor) -> th.Tensor:
        '''
        Forward propagation
        :param observations: (dict) изображение; координаты и углы ориентации агентов
        :return: features tensor
        '''
        encoded_tensor_list = []

        for key, extractor in self.extractors.items():
            encoded_tensor_list.append(extractor(observations[key]))

        return th.cat(encoded_tensor_list, dim=1)

    
class ResBlock(nn.Module):
    def __init__(self, n_filters, kernel_size):
        """
        Инициализация кастомного резнетовского блока
        :param n_filters: (int) количество фильтров сверточного слоя
        :param kernel_size: (int) размер ядра свертки
        """
        super().__init__()
        self.n_filters = n_filters
        self.kernel_size = kernel_size

        self.b1 = nn.Conv2d(self.n_filters, self.n_filters, self.kernel_size, padding='same')
    
        self.b2 = nn.BatchNorm2d(self.n_filters, eps = 0.001, momentum= 0.99)
        self.b3 = nn.Conv2d(self.n_filters, self.n_filters, self.kernel_size, padding='same')
        self.b4 = nn.BatchNorm2d(self.n_filters, eps = 0.001, momentum= 0.99)
        
    def forward(self, x):
        '''
        Forward propagation
        :param x: input
        :return: output
        '''
        residual = x
        y = F.relu(self.b1(x))
        y = self.b2(y)
        y = F.relu(self.b3(y))
        y = self.b4(y)
        y += residual
        y = F.relu(y)
        return y

## Environment gym class

In [ ]:
class CustomEnv(gym.Env):
    '''
    Оборочивание класса среды в среду gym
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self, obstacle_turn: bool, Total_war: bool, num_obs, num_enemy: int, inp_dim: int,
                 size_obs, steps_limit, vizualaze = False, head_velocity = 0.01, rew_col = -70,
                 rew_win = 100, rew_defeat = -100 , EnemyLidSet = [120, 120] , AllyLidSet = [50, 90]):
        '''
        Инициализация класса среды
        :param obstacle_turn: (bool) Флаг генерации препятствий
        :param vizualaze: (bool) Флаг генерации препятствий
        :param Total_war: (bool) Флаг режима игры (с противником или без)
        :param steps_limit: (int) Максимальное количество действий в среде за одну игру
        '''
        self.log_koef = 50
        self.ang_Norm_coef = np.pi
        self.coords_Norm_coef = 500
        self.inp_dim = inp_dim
        
        # optionEnemy = [120, 120]     # настройки противника [0] - дальность СТЗ; [1] - угол СТЗ (градусы)
        # optionAlie = [50, 90]        # настройки союзника [0] - дальность СТЗ; [1] - угол СТЗ (градусы)
        
        self.obstacle_turn = obstacle_turn
        self.Total_war = Total_war
        self.num_obs = num_obs
        self.num_enemy = num_enemy
        self.size_obs = size_obs
        self.steps_limit = steps_limit
        self.vizualaze = vizualaze
        self.head_velocity = head_velocity
        self.rew_col = rew_col
        self.rew_win = rew_win
        self.rew_defeat = rew_defeat
        self.EnemyLidSet = EnemyLidSet
        self.AllyLidSet = AllyLidSet
        
        self.enviroment = Enviroment(self.obstacle_turn, self.vizualaze, self.Total_war,
                                     self.head_velocity, self.num_obs, self.num_enemy, 
                                     self.size_obs, self.steps_limit, self.rew_col, 
                                     self.rew_win, self.rew_defeat, epsilon = 100,
                                     sigma = 30, optionEnemy = self.EnemyLidSet, optionAlie = self.AllyLidSet)


        self.action_space = spaces.Discrete(8)
        self.observation_space = gym.spaces.Dict({
                    'img': spaces.Box(low=0, high=255, shape=(self.inp_dim, self.inp_dim, 3), dtype=np.uint8),
                    'posRobot': spaces.Box(low=np.array([0, 0,-3.14]), high=np.array([500, 500, 3.14])),
                    'target':   spaces.Box(low=np.array([0, 0,-3.14]), high = np.array([500, 500, 3.14]))
                                                })

        state = self.enviroment.reset()
        
        self.img1 = state.img
        self.img2 = state.img
        self.img3 = state.img
        self.Img = None
        

    def make_layers(self):
        """
        Функция наслоения изображений трех последовательных шагов в среде
        :param img1, img2, img3: состояния среды на трех последовательных шагах
        :return: new_img: изображение, содержащее информацию о состояниях среды на трех последовательных шагах, отображенную с разной интенсивностью
        """
        new_img = cv2.addWeighted(self.img2, 0.4, self.img1, 0.2, 0)
        self.Img = cv2.addWeighted(self.img3, 0.7, new_img, 0.5, 0)
    
    
    def step(self, action):
        """
        Метод осуществления шага в среде
        :param action: (int) направление движения в среде
        :return: dict_state, reward, not done, {}: состояние, реворд, флаг терминального состояния, информация о среде
        """
        
        state, reward, done, numstep = self.enviroment.step(action)
        # state.img = cv2.resize(state.img, (self.inp_dim,self.inp_dim))
        
        self.img1 = self.img2
        self.img2 = self.img3
        self.img3 = state.img
        
        self.make_layers()
    
        x2 = state.posRobot[0]
        y2 = state.posRobot[1]
    
        x4 = state.target[0,0]
        y4 = state.target[0,1]
        
        
        f2 =  state.target[0,2]
        f2 = np.deg2rad(f2)
        
        Ax4, Ay4 = -np.cos(f2), np.sin(f2)
        Bx24, By24 = x2 - x4, y2 - y4

        dist = - np.sqrt(np.abs((x2-x4)**2 + (y2-y4)**2))
        phy = (Ax4*Bx24 + Ay4*By24)/(np.sqrt(Ax4**2 + Ay4**2) * np.sqrt(Bx24**2 + By24**2))
        reward_l = phy*(dist+500) * 0.01 * (not done) + np.round(reward, 2).sum()

        
        dict_state = {'img':     self.Img,  
                      'posRobot':self.normPoseRobot(state.posRobot),  
                      'target':  self.normTarget(state.target).reshape(-1)}

        return dict_state, reward_l, done, {}
    
    def normTarget(self, coords):
        '''
        Метод нормализации координат
        :return: coords: нормализованные координаты
        '''
        coords=np.float32(coords)
        coords[:,2]  = coords[:,2] / self.ang_Norm_coef #угол
        coords[:,:2] = coords[:,:2] / self.coords_Norm_coef #координаты
        
        return coords

    def normPoseRobot(self, coords):
        '''
        Метод нормализации координат
        :return: coords: нормализованные координаты
        '''
        coords=np.float32(coords)
        coords[2]  = coords[2] / self.ang_Norm_coef #угол
        coords[:2] = coords[:2] / self.coords_Norm_coef #координаты
        
        return coords


    def reset(self):
        '''
        Метод обновления игры
        :return: dict_state: состояние
        '''
                
        state = self.enviroment.reset()
        # state.img = cv2.resize(state.img, (self.inp_dim,self.inp_dim))
        
        self.img2 = state.img
        self.img3 = state.img
        
        dict_state = {'img':     state.img,  
                      'posRobot':self.normPoseRobot(state.posRobot),  
                      'target':  self.normTarget(state.target).reshape(-1)}

        return dict_state

    def render(self, model, num_gifs=1):
        '''
        Метод вывода информации об игре
        :param mode:
        :return:
        '''
        for i in range(num_gifs):
            
            images = []
            obs = self.reset()
            img = obs['img']# env.render(mode='rgb_array')
            done = False
                
            height, width, layers = img.shape
            size = (width,height)
            out = cv2.VideoWriter(f"video{i}.avi",cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            out.write(img)
            while not done:

                action, _ = model.predict(obs)
                obs, _, done ,_ = self.step(action)
                img = obs['img']
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                out.write(img)
            out.release()
    
    def get_statistic(self, model, num_games):
        collision = 0
        win = 0
        destroyed = 0
        loss = 0
        
        pbar = tqdm(range(num_games))
        for i in pbar:
            obs = self.reset()
            done = False
            while not done:
                action, _ = model.predict(obs)
                obs, reward, done ,_ = self.step(action)
                
            if reward == self.rew_col:#win
                collision+=1
            elif reward == self.rew_win:# loss
                win +=1
            elif reward == self.rew_defeat:# loss
                destroyed +=1
            else:    #not_achieved
                loss+=1
        
        print("Win: ",win/num_games)
        print("destroyed: ", destroyed/num_games)
        print("loss: ",loss/num_games)
        print("collision: ",collision/num_games)

In [ ]:
env = CustomEnv(obstacle_turn = True,
                vizualaze     = False, 
                Total_war     = True,
                inp_dim       = 500,
                head_velocity = 0.005,#0.005
                num_obs       = 5, 
                num_enemy     = 1, 
                size_obs      = [50, 60],
                rew_col       = -70,
                rew_win       = 100,
                rew_defeat    = -100,
                steps_limit   = 2000,
                EnemyLidSet   = [45,90],
                AllyLidSet    = [40, 90])

## Initialize the agent

In [ ]:
from stable_baselines3 import DQN, PPO, A2C

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=512+6),
    activation_fn=torch.nn.ReLU,
    net_arch = [dict(pi=[134, 32, 8], vf=[134, 32, 8])])

# model = DQN(policy                  = 'MultiInputPolicy',  # 1 neural network metod
#             env                     = env,
#             learning_rate           = 0.0001,
#             buffer_size             = 10000,
#             batch_size              = 20,
#             gamma                   = 0.99,
#             tensorboard_log         = "./tensorboard_logs_disc_mult/",
#             policy_kwargs           = policy_kwargs,
#             verbose                 = 0,
#             device                  = 'cuda')

model = PPO(policy          = 'MultiInputPolicy',  # 2 neural networks metod
            env             = env,
            learning_rate   = 0.001,
            n_steps         = 64,
            batch_size      = 64,
            gamma           = 0.99,
            gae_lambda      = 0.95,
            tensorboard_log = "./tensorboard_logs/",
            policy_kwargs   = policy_kwargs,
            verbose         = 1,
            device          = 'cuda')

# model = A2C(policy          = 'MlpPolicy',  # 2 neural networks metod
#             env             = env,
#             learning_rate   = 0.0001,
#             n_steps         = 10,
#             gamma           = 0.99,
#             gae_lambda      = 0.95,
#             tensorboard_log = "./tensorboard_logs_disc_mult/",
#             policy_kwargs   = policy_kwargs,
#             verbose         = 0,
#             device          = 'cuda')

## Make callback

In [ ]:
log_dir = './saved_models/PPO/callback/'  # For A2C agent: './saved_models/A2C/callback'
os.makedirs(log_dir, exist_ok=True)
env = Monitor(env, log_dir)
callback = SaveOnBestTrainingRewardCallback(check_freq=5000, log_dir=log_dir)

## Learn model

In [ ]:
model.learn(total_timesteps=5e5,callback=callback)

## Load the best model and get statistics

In [ ]:
# path = './saved_models/PPO/callback0/best_model_PPO/'  # For A2C agent: './saved_models/A2C/callback0/best_model_A2C/'
# model = PPO.load(path, env=env_test,device = 'cuda')  # For A2C agent: A2C.load(path, env=env_test)

In [ ]:
# env.get_statistic(model, 1000)

## Check tensorboard

In [ ]:
# !tensorboard --logdir ./tensorboard_logs_discr/